## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
# city_data_df = pd.read_csv("WeatherPy_database.csv")
# city_data_df.head()

city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description
0,0,Bluff,NZ,2022-03-05 18:54:38,-46.6000,168.3333,64.80,71,93,14.85,overcast clouds
1,1,Busselton,AU,2022-03-05 18:54:39,-33.6500,115.3333,66.04,73,0,11.86,clear sky
2,2,Khatanga,RU,2022-03-05 18:54:39,71.9667,102.5000,-22.95,99,99,7.20,overcast clouds
3,3,Leh,IN,2022-03-05 18:54:39,34.1667,77.5833,9.41,65,85,2.04,overcast clouds
4,4,Bredasdorp,ZA,2022-03-05 18:54:40,-34.5322,20.0403,71.28,73,0,13.80,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description
10,10,Harper,LR,2022-03-05 18:52:32,4.3750,-7.7169,80.71,83,9,7.78,clear sky
11,11,Vaini,TO,2022-03-05 18:54:42,-21.2000,-175.2000,78.96,94,75,6.91,broken clouds
15,15,Vanimo,PG,2022-03-05 18:53:48,-2.6741,141.3028,76.86,91,100,6.58,overcast clouds
17,17,Huarmey,PE,2022-03-05 18:52:35,-10.0681,-78.1522,78.60,67,37,11.65,scattered clouds
18,18,Fare,PF,2022-03-05 18:47:14,-16.7000,-151.0167,81.81,73,3,19.44,light rain
22,22,Rikitea,PF,2022-03-05 18:54:45,-23.1203,-134.9692,79.66,76,11,22.59,light rain
23,23,Boa Vista,BR,2022-03-05 18:47:03,2.8197,-60.6733,80.58,74,75,4.61,light rain
24,24,Merauke,ID,2022-03-05 18:52:42,-8.4667,140.3333,77.50,90,100,6.06,moderate rain
27,27,San Rafael,AR,2022-03-05 18:54:46,-34.6177,-68.3301,76.51,27,34,11.74,scattered clouds
29,29,Avarua,CK,2022-03-05 18:54:47,-21.2078,-159.7750,80.65,83,75,10.36,broken clouds


In [28]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                200
City                   200
Country                198
Date                   200
Lat                    200
Lng                    200
Max Temp               200
Humidity               200
Cloudiness             200
Wind Speed             200
Weather description    200
dtype: int64

In [27]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [30]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
10,Harper,LR,80.71,clear sky,4.3750,-7.7169,
11,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,
15,Vanimo,PG,76.86,overcast clouds,-2.6741,141.3028,
17,Huarmey,PE,78.60,scattered clouds,-10.0681,-78.1522,
18,Fare,PF,81.81,light rain,-16.7000,-151.0167,
22,Rikitea,PF,79.66,light rain,-23.1203,-134.9692,
23,Boa Vista,BR,80.58,light rain,2.8197,-60.6733,
24,Merauke,ID,77.50,moderate rain,-8.4667,140.3333,
27,San Rafael,AR,76.51,scattered clouds,-34.6177,-68.3301,
29,Avarua,CK,80.65,broken clouds,-21.2078,-159.7750,


In [36]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json() 
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [39]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna()

In [40]:
# 8a. Create the output File (CSV)
output_data_file="WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

clean_hotel_df.head()

,City,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
10,Harper,LR,80.71,clear sky,4.3750,-7.7169,Screensaver
11,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
15,Vanimo,PG,76.86,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
17,Huarmey,PE,78.60,scattered clouds,-10.0681,-78.1522,Hotel Miramar
18,Fare,PF,81.81,light rain,-16.7000,-151.0167,Maitai Lapita Village Huahine


In [43]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Weather description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [44]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# 11b. Display the figure

fig = gmaps.figure(center=(35.0, 32.0), zoom_level=1.8)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))